In [1]:
import sys
sys.path.insert(1, "../")
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from yolov2 import YOLOv2D19 as YOLOv2
from detection_datasets import VOCDatasetV2
from torch import optim
from loss import YOLOv2Loss
from train import *
import torch.optim.lr_scheduler as lr_scheduler
from data_preprocessing import get_norms
import pickle
with open('anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

In [2]:
device = torch.device('cuda:0')
dtype=torch.float32

In [3]:
norms = get_norms('../../datasets/VOCdevkit/trainval_norms.json')
means = norms['means']
stds = norms['stds']

In [4]:
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
import gc

# Invoke garbage collector
gc.collect()

# Clear GPU cache
torch.cuda.empty_cache()
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

Allocated memory: 0.00 MB
Allocated memory: 0.00 MB


In [5]:
transforms = A.Compose([
    A.Resize(width=416, height=416),
    A.VerticalFlip(p=1.0),
    A.Normalize(mean=means, std=stds),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', 
                         subsets = [('VOC2012', 'trainval')],
                         scales=[13], anchors=anchors, transforms=transforms, 
                         dtype=dtype, device=device)
val_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', 
                       subsets = [('VOC2007', 'val')],
                       scales=[13], anchors=anchors, transforms=transforms, 
                       dtype=dtype, device=device)

True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\val.txt


In [6]:
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False)

In [7]:
loss_fn = YOLOv2Loss(anchors=anchors)

In [8]:
model = YOLOv2(device=device, dtype=dtype)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Models\yolov2.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location=self.devic

In [9]:
epochs=1
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [10]:
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=1.0)

In [ ]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../log/YOLOv2/training/')

2024-12-22 21:09:36.418164 Epoch 1 
2024-12-22 21:09:38.707747 Batch 1 
Before inference 2024-12-22 21:09:38.748585
After inference 2024-12-22 21:10:06.838459
Loss start 2024-12-22 21:10:06.845463
Conf start 2024-12-22 21:10:06.845463
Box start 2024-12-22 21:10:06.925161
Class start 2024-12-22 21:10:06.965458
Sum everything 2024-12-22 21:10:07.026506
After loss fn 2024-12-22 21:10:07.035632
After scaler-loss backward 2024-12-22 21:11:07.895611
After unscaling 2024-12-22 21:11:08.035497
After scaler step 2024-12-22 21:11:08.038633
After scaler update 2024-12-22 21:11:08.046458
2024-12-22 21:11:09.942235 Batch 2 
Before inference 2024-12-22 21:11:10.215371


In [12]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../log/YOLOv2/training/')

2024-12-22 20:45:13.981076 Epoch 1 
2024-12-22 20:45:14.996489 Batch 1 
Before inference 2024-12-22 20:45:15.015560
After inference 2024-12-22 20:45:15.566507
Loss start 2024-12-22 20:45:15.567513
Conf start 2024-12-22 20:45:15.567513
After loss fn 2024-12-22 20:45:15.606631
After scaler-loss backward 2024-12-22 20:45:18.043209
After unscaling 2024-12-22 20:45:18.059567
After scaler step 2024-12-22 20:45:18.059567
After scaler update 2024-12-22 20:45:18.059567
2024-12-22 20:45:18.697031 Batch 2 
Before inference 2024-12-22 20:45:18.697031
After inference 2024-12-22 20:45:18.856477
Loss start 2024-12-22 20:45:18.856477
Conf start 2024-12-22 20:45:18.856477
After loss fn 2024-12-22 20:45:18.856477
After scaler-loss backward 2024-12-22 20:45:21.095372
After unscaling 2024-12-22 20:45:21.095372
After scaler step 2024-12-22 20:45:21.095372
After scaler update 2024-12-22 20:45:21.095372
2024-12-22 20:45:21.743539 Batch 3 
Before inference 2024-12-22 20:45:21.744565
After inference 2024-12-22

KeyboardInterrupt: 

In [12]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../log/YOLOv2/training/')

2024-12-22 20:43:35.379370 Epoch 1 
2024-12-22 20:43:36.406915 Batch 1 
Before inference 2024-12-22 20:43:36.425448
After inference 2024-12-22 20:43:36.980048
Loss start 2024-12-22 20:43:36.980048
Conf start 2024-12-22 20:43:36.980048
Box start 2024-12-22 20:43:37.018984
Class start 2024-12-22 20:43:37.043258
Sum everything 2024-12-22 20:43:37.067808
After loss fn 2024-12-22 20:43:37.074276
After scaler-loss backward 2024-12-22 20:43:39.867387
After unscaling 2024-12-22 20:43:39.887851
After scaler step 2024-12-22 20:43:39.887851
After scaler update 2024-12-22 20:43:39.888851
2024-12-22 20:43:40.670400 Batch 2 
Before inference 2024-12-22 20:43:40.670400
After inference 2024-12-22 20:43:40.833210
Loss start 2024-12-22 20:43:40.833210
Conf start 2024-12-22 20:43:40.833210
Box start 2024-12-22 20:43:40.833210
Class start 2024-12-22 20:43:40.834715
Sum everything 2024-12-22 20:43:40.835227
After loss fn 2024-12-22 20:43:40.835227
After scaler-loss backward 2024-12-22 20:43:43.345190
After

KeyboardInterrupt: 